# GANs (Generative Adversarial Networks)

In [80]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [81]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/mnist', one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


### Hyperparameters

In [100]:
# Modeling
num_input = 28 * 28
num_hidden = 256
num_noise = 128 # 생성기의 입력값으로 사용할 노이즈의 크기

# Learning
total_epoch = 30
batch_size = 100
learning_rate = 0.002

# Evaluation
num_samples = 10

### weight와 bias를 정의해 줍니다.

GANs도 unsupervised learning의 하나이기 때문에 label이 없으므로 y_true 필요 없습니다.

단, noise z_true를 입력값으로 사용합니다. 

In [83]:
x_true = tf.placeholder(tf.float32, [None, num_input])
z_true = tf.placeholder(tf.float32, [None, num_noise])

Generator NN 에 사용하는 변수들을 정의해 줍니다.

In [84]:
weight_generator_1 = tf.Variable(tf.truncated_normal([num_noise, num_hidden], stddev=0.01))
bias_generator_1 = tf.Variable(tf.zeros([num_hidden]))

weight_generator_2 = tf.Variable(tf.truncated_normal([num_hidden, num_input], stddev=0.01))
bias_generator_2 = tf.Variable(tf.zeros([num_input]))

Discriminator NN에 사용하는 변수들을 정의해 줍니다. 

이때 Discriminator의 최종 결과값은 진짜와 얼마나 가깝느냐를 판단하기 위한 1개의 scalar value임에 주목합니다!

In [85]:
weight_discriminator_1 = tf.Variable(tf.truncated_normal([num_input, num_hidden], stddev=0.01))
bias_discriminator_1 = tf.Variable(tf.zeros([num_hidden]))

weight_discriminator_2 = tf.Variable(tf.truncated_normal([num_hidden, 1], stddev=0.01)) # 1임에 주목!
bias_discriminator_2 = tf.Variable(tf.zeros([1])) # 1임에 주목!

### Modeling

Generator NN을 구성합니다.

In [86]:
def generator(noise_z):
    hidden_layer = tf.nn.relu(tf.matmul(noise_z, weight_generator_1) + bias_generator_1)
    output_generator = tf.sigmoid(tf.matmul(hidden_layer, weight_generator_2) + bias_generator_2)
    return output_generator

Distinguisher NN을 구성합니다.

In [87]:
def discriminator(inputs):
    hidden_layer = tf.nn.relu(tf.matmul(inputs, weight_discriminator_1) + bias_discriminator_1)
    output_discriminator = tf.sigmoid(tf.matmul(hidden_layer, weight_discriminator_2) + bias_discriminator_2)
    return output_discriminator

Random noise (z_true) 를 만들어 줍니다.

In [88]:
def get_noise(batch_size):
    return np.random.normal(size=(batch_size, num_noise)) # (100,128)

노이즈를 이용해 랜덤한 이미지를 생성합니다.

In [89]:
G = generator(z_true) # (None, 784)

노이즈를 이용해 생성한 이미지가 진짜 이미지인지 판별한 값을 구합니다.

In [90]:
D_generator = discriminator(G)

진짜 이미지를 이용해 판결한 값을 구합니다.

In [91]:
D_real = discriminator(x_true)

* GANs 모델의 최적화는 loss_G와 loss_D를 최대화하는 것입니다.
* loss_D를 최대화하기 위해서는 D_generator를 최소화하게 됩니다. 
* Discriminator에 진짜 이미지를 넣었을 때에도 최대값을 tf.log(D_real), 가짜 이미지를 넣었을 때에도 최대값을 tf.log(1-D_generator) 을 갖도록 학습시키기 때문입니다.

Discriminator는 Generator가 만들어낸 이미지가 가짜라고 판단하도록 discriminator NN을 학습시킵니다.

In [92]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_generator))

반면, loss_G를 최대화하기 위해서는 D_generator 값을 최대화하게 되는데, 이것은 가짜 이미지를 넣었을 때 discriminator가 최대한 실제 이미지라고 판단하도록 generator NN을 학습시킵니다. 논문에서는 loss_D와 같은 수식으로 최소화하는 generator를 찾지만, 결국 D_generator 값을 최대화하는 것이므로 다음과 같이 사용할 수 있습니다.

In [93]:
loss_G = tf.reduce_mean(tf.log(D_generator))

loss_D를 구할 때에는 generator NN에 사용되는 변수만 사용하여 최적화를 하고, loss_G를 구할 때에는 discriminator NN에 사용되는 변수만 사용하여 최적화를 합니다. 

In [94]:
D_var_list = [weight_discriminator_1, bias_discriminator_1, weight_discriminator_2, bias_discriminator_2]
G_var_list = [weight_generator_1, bias_generator_1, weight_generator_2, bias_generator_2]

GANs 논문의 수식에 따르면 loss를 극대화해야 하지만, minimize하는 최적화함수를 사용하기 때문에 최적화하려는 loss_D와 loss_G에 음수 부호를 붙여 줍니다.

In [95]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

### Learning

In [96]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [97]:
total_batch = int(mnist.train.num_examples/batch_size)

In [98]:
loss_value_Discriminator = 0
loss_value_Generator = 0

In [101]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size)

        _, loss_value_Discriminator = sess.run([train_D, loss_D], {x_true:batch_xs,z_true:noise})
        _, loss_value_Generator = sess.run([train_G, loss_G], {z_true:noise})
    print "Epoch : {0}, Loss (Discriminator) : {1}, Loss (Generator) : {2}"\
            .format(epoch + 1, loss_value_Discriminator, loss_value_Generator)
    
    # 학습 과정을 보여주기 위해 주기적으로 이미지를 생성하여 저장합니다.
    noise = get_noise(num_samples)
    samples = sess.run(G, {z_true: noise})
    fig, ax = plt.subplots(1, num_samples, figsize=(num_samples,1))
    
    for i in range(num_samples):
        ax[i].set_axis_off
        ax[i].imshow(np.reshape(samples[i], (28,28)))
        
    plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
    plt.close(fig)

Epoch : 1, Loss (Discriminator) : -1.9008198251e-06, Loss (Generator) : -15.3966360092
Epoch : 2, Loss (Discriminator) : -2.17089927901e-06, Loss (Generator) : -16.0739517212
Epoch : 3, Loss (Discriminator) : -4.69089911803e-07, Loss (Generator) : -16.3357448578
Epoch : 4, Loss (Discriminator) : -2.8553527045e-06, Loss (Generator) : -16.9084358215
Epoch : 5, Loss (Discriminator) : -2.9385171274e-07, Loss (Generator) : -17.3470153809
Epoch : 6, Loss (Discriminator) : -5.53731865693e-07, Loss (Generator) : -17.641412735
Epoch : 7, Loss (Discriminator) : -2.53928442362e-06, Loss (Generator) : -18.0438404083
Epoch : 8, Loss (Discriminator) : -2.11596855593e-07, Loss (Generator) : -16.8216533661
Epoch : 9, Loss (Discriminator) : -1.38629484177, Loss (Generator) : -0.693133115768
Epoch : 10, Loss (Discriminator) : -1.38629484177, Loss (Generator) : -0.693147420883
Epoch : 11, Loss (Discriminator) : -1.38629484177, Loss (Generator) : -0.693147420883
Epoch : 12, Loss (Discriminator) : -1.38629